In [128]:
import skeleton as s
import numpy as np

In [143]:
init_pos = np.random.uniform(low=0, high=1.0, size=(2, 2))
init_vel = np.random.uniform(low=-430, high=430, size=(2, 2))
time_step = 3
m = 6.6335 * pow(10, -26) #mass of argon atom in kg
n = 2 #number of particles
number_of_steps = 100

In [144]:
print("initial pos\n", init_pos,"\n initial vel\n" ,init_vel)

initial pos
 [[0.20401175 0.30884553]
 [0.21400961 0.75029372]] 
 initial vel
 [[ 388.78641596  -31.15490098]
 [  42.57949927 -280.39737919]]


In [155]:
#compute the forces on the particles at each timestep
def lj_force(position, n):
    e = 1.65 *pow(10,-21)
    s = 3.405 *pow(10,-21)
    F = 0
    pos = position[:,-n:]
    for j in range(n):
        f = 0    # force on the j-th particle
        for i in range(n-1):
            if i+j+1 >= n: #take into account particles 0,1,....,j-1
                rel_pos=pos[:,j+i+1-n]-pos[:,j] #gives position of i-th particle with resprect to j-th particle (points from i to j)
                r = np.linalg.norm(pos[:,j+i+1-n]-pos[:,j])
                f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # ff on the j-th part. from the interaction with the i-th part.
            else: #take into account particles j+1,....,n-1
                rel_pos=pos[:,j+i+1]-pos[:,j] #gives position of i-th particle with resprect to j-th particle (points from i to j)
                r = np.linalg.norm(pos[:,j+i+1]-pos[:,j])
                f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # f on the j-th part. from the interaction with the i-th part.
        if j == 0:
            F = f
        else:
            F = np.concatenate((F, f), axis=0, out=None)
    
    F_matrix = np.zeros((2,n)) #2 is the number of dimensions, n is the number of particles
    F_matrix[:n] = F[0:n]
    F_matrix[1:] = F[2:]
    return F_matrix

In [156]:
final_matrix_pos = np.copy(init_pos)
final_matrix_vel = np.copy(init_vel)

def euler(final_matrix_pos, final_matrix_vel):
    
    latest_pos = np.copy(final_matrix_pos[:,-2:])          #the -2 will eventually become -(number of particles)
    latest_vel = np.copy(final_matrix_vel[:,-2:])
    
    new_latest_pos = latest_pos + latest_vel * time_step
    new_latest_vel = latest_vel + 1/m * lj_force(latest_pos, 2)
    return new_latest_pos, new_latest_vel


In [157]:
def kin_en(v,n): #v is the last step velocity and n the number of particles
    K = 0
    for i in range(n):
        K += 0.5*m*((v[0,i]*v[0,i])+(v[1,i]*v[1,i]))
    return K

In [158]:
e = 1.65*pow(10,-21)
s = 3.405*pow(10,-10)
def pot_en(position, n): #position is the matrix with all the positions stored in it, n is the number of particles
    U = 0;
    pos = position[:,-n:]
    for j in range(n):
        for i in range(n-1):
            if i+j+1 >= n: #take into account particles 0,1,....,j-1
                r = np.linalg.norm(pos[:,j+i+1-n]-pos[:,j])
                U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potnetial energy of the j-th part. from the interaction with the i-th part.
            else: #take into account particles j+1,....,n-1
                r = np.linalg.norm(pos[:,j+i+1]-pos[:,j])
                U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potential energy of the j-th part. from the interaction with the i-th part.
                
    return U

In [159]:
#Create a 2x8 matrix to store the velocity of each particle at each step in time.
next_step_velocity = np.copy(init_vel)

#Create a 2x8 matrix to store the position of each particle at each step in time.
next_step_position = np.copy(init_pos)

final_vector_energy = np.array([kin_en(init_vel, n) + pot_en(final_matrix_pos, n)])

for i in range(number_of_steps):
    next_step_position, next_step_velocity = euler(final_matrix_pos, final_matrix_vel)
    for k in range(2): #implement boundary conditions
        for j in range(2):
            next_step_position[k, j] = next_step_position[k, j]%1
    final_matrix_pos =  np.concatenate((final_matrix_pos, next_step_position), axis=1, out=None)
    final_matrix_vel =  np.concatenate((final_matrix_vel, next_step_velocity), axis=1, out=None)
    final_vector_energy = np.concatenate((final_vector_energy, np.array([kin_en(next_step_velocity, n) + pot_en(final_matrix_pos, n)])), axis=0, out=None)

print("Positions:\n" , final_matrix_pos)
print("Velocities:\n" , final_matrix_vel)
print("Energy:\n" , final_vector_energy)

Positions:
 [[0.20401175 0.30884553 0.56325964 0.84414258 0.         0.
  0.         0.                nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        na

<ipython-input-158-02ff9ef1b671>:13: RuntimeWarning: divide by zero encountered in double_scalars
  U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potential energy of the j-th part. from the interaction with the i-th part.
<ipython-input-158-02ff9ef1b671>:10: RuntimeWarning: divide by zero encountered in double_scalars
  U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potnetial energy of the j-th part. from the interaction with the i-th part.
<ipython-input-155-2aeab00c90d8>:17: RuntimeWarning: divide by zero encountered in double_scalars
  f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # f on the j-th part. from the interaction with the i-th part.
<ipython-input-155-2aeab00c90d8>:17: RuntimeWarning: invalid value encountered in multiply
  f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # f on the j-th part. from the interaction with the i-th part.
<ipython-input-155-2aeab00c90d8>:13: RuntimeWarning: invalid value encountered in multiply
  f += rel_pos*((24*e/pow(